## DIA 086: Agregar Filtro por Fecha al Historial de Predicciones en Flask

Hoy aprenderás a mejorar el historial de predicciones permitiendo filtrar por rango de fechas. Esto es muy útil para analizar predicciones hechas en un período específico (por día, semana o mes).

✅ ¿Qué aprenderás hoy?

📌 Agregar un formulario de filtro de fechas en HTML.

📌 Procesar fechas desde el formulario.

📌 Consultar la base de datos con condiciones usando SQLAlchemy.

🛠️ 1. Estructura del Proyecto (igual)
pgsql
Copiar
Editar
titanic_app/
├── app.py
├── model.pkl
├── db.sqlite3
└── templates/
    ├── index.html
    └── historial.html  ✅ Modificado
🧩 2. Código Flask Actualizado (app.py)
Agregamos soporte para filtros GET (/historial?desde=...&hasta=...):

python
Copiar
Editar
from flask import Flask, render_template, request, send_file
from flask_sqlalchemy import SQLAlchemy
import joblib
import numpy as np
import datetime
import pandas as pd
import io

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///db.sqlite3'
db = SQLAlchemy(app)

class Registro(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    f1 = db.Column(db.Float)
    f2 = db.Column(db.Float)
    f3 = db.Column(db.Float)
    f4 = db.Column(db.Float)
    prediccion = db.Column(db.String(50))
    fecha = db.Column(db.DateTime, default=datetime.datetime.utcnow)

modelo = joblib.load("model.pkl")

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/predecir", methods=["POST"])
def predecir():
    try:
        f1 = float(request.form["f1"])
        f2 = float(request.form["f2"])
        f3 = float(request.form["f3"])
        f4 = float(request.form["f4"])

        datos = np.array([[f1, f2, f3, f4]])
        pred = modelo.predict(datos)[0]

        nuevo_registro = Registro(f1=f1, f2=f2, f3=f3, f4=f4, prediccion=str(pred))
        db.session.add(nuevo_registro)
        db.session.commit()

        return render_template("index.html", resultado=f"✅ Resultado: Clase {pred}")
    except:
        return render_template("index.html", resultado="❌ Error en los datos")

@app.route("/historial")
def historial():
    desde = request.args.get("desde")
    hasta = request.args.get("hasta")

    query = Registro.query

    if desde:
        desde_dt = datetime.datetime.strptime(desde, "%Y-%m-%d")
        query = query.filter(Registro.fecha >= desde_dt)
    if hasta:
        hasta_dt = datetime.datetime.strptime(hasta, "%Y-%m-%d") + datetime.timedelta(days=1)
        query = query.filter(Registro.fecha < hasta_dt)

    registros = query.order_by(Registro.fecha.desc()).all()
    return render_template("historial.html", registros=registros, desde=desde, hasta=hasta)

@app.route("/exportar")
def exportar():
    registros = Registro.query.order_by(Registro.fecha.desc()).all()
    data = [{
        "ID": r.id,
        "Feature 1": r.f1,
        "Feature 2": r.f2,
        "Feature 3": r.f3,
        "Feature 4": r.f4,
        "Predicción": r.prediccion,
        "Fecha": r.fecha.strftime('%Y-%m-%d %H:%M:%S')
    } for r in registros]

    df = pd.DataFrame(data)
    output = io.StringIO()
    df.to_csv(output, index=False)
    output.seek(0)

    return send_file(
        io.BytesIO(output.getvalue().encode()),
        mimetype='text/csv',
        as_attachment=True,
        download_name='historial_predicciones.csv'
    )

if __name__ == "__main__":
    db.create_all()
    app.run(debug=True)
🖼️ 3. Formulario HTML para Filtro (templates/historial.html)
Agrega esto encima de la tabla:

html
Copiar
Editar
<form method="get" class="row g-3 mb-4">
    <div class="col-auto">
        <label for="desde" class="form-label">Desde:</label>
        <input type="date" name="desde" class="form-control" value="{{ desde }}">
    </div>
    <div class="col-auto">
        <label for="hasta" class="form-label">Hasta:</label>
        <input type="date" name="hasta" class="form-control" value="{{ hasta }}">
    </div>
    <div class="col-auto d-flex align-items-end">
        <button type="submit" class="btn btn-primary">Filtrar</button>
    </div>
</form>
🔁 Esto mantiene los valores seleccionados al volver a cargar la página.

🔍 Líneas Clave Explicadas
request.args.get("desde"): Captura parámetros del filtro.

query.filter(...): Filtra los registros por fecha usando SQLAlchemy.

datetime.timedelta(days=1): Asegura que la fecha "hasta" incluya el día completo.

value="{{ desde }}": Mantiene el valor en el campo tras aplicar el filtro.